第一版deep-learning http://neuralnetworksanddeeplearning.com/chap1.html

In [ ]:
import random
import numpy as np

In [69]:
class network(object):
    
    #一个[2,3,1]的网络，第一层是输入层，没有biase,剩下两层biase的机构是3*1,2*1的矩阵。
    #weights是两个3*2， 3*1的矩阵。
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x,y in sizes[:-1], sizes[1:]]
    
    # a -> input
    def forward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, training_data,epochs, mini_batch_size, eta, test_data=None):
        if test_data:n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k : k + mini_batch_size] for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "EPOCH {0} : {1} / {2}".format(j, self.evaluate(test_data), n_test)
            else:
                print "EPOCH {0} complete".format(j)
    
    #
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x,y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.biases = [(b - eta/len(mini_batch)*nb) for b, nb in zip(self.biases, nabla_b)]
        self.weights = [(w - eta/len(mini_batch)*nw) for w, nw in zip(self.weights, nabla_w)]
        
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        #forward
        activation = x
        activations = [x]
        zs = []
        for b,w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        #backward
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        #此处的layer与普通记号不同，1=the last layer,2 = second last layer...and so no
        #l加了负号，本质是一样的
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.forward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)
    
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [70]:
net =network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

EPOCH 0 : 8304 / 10000
EPOCH 1 : 8396 / 10000
EPOCH 2 : 8415 / 10000
EPOCH 3 : 8481 / 10000
EPOCH 4 : 8518 / 10000
EPOCH 5 : 8544 / 10000
EPOCH 6 : 8537 / 10000
EPOCH 7 : 8573 / 10000
EPOCH 8 : 8572 / 10000
EPOCH 9 : 8573 / 10000
EPOCH 10 : 8571 / 10000
EPOCH 11 : 8568 / 10000
EPOCH 12 : 8555 / 10000
EPOCH 13 : 8604 / 10000
EPOCH 14 : 8578 / 10000
EPOCH 15 : 8590 / 10000
EPOCH 16 : 8604 / 10000
EPOCH 17 : 8594 / 10000
EPOCH 18 : 8612 / 10000
EPOCH 19 : 8598 / 10000
EPOCH 20 : 8599 / 10000
EPOCH 21 : 8736 / 10000
EPOCH 22 : 9439 / 10000
EPOCH 23 : 9424 / 10000
EPOCH 24 : 9443 / 10000
EPOCH 25 : 9440 / 10000
EPOCH 26 : 9431 / 10000
EPOCH 27 : 9443 / 10000
EPOCH 28 : 9424 / 10000
EPOCH 29 : 9432 / 10000


In [22]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import cPickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [19]:
training_data, validation_data, test_data = load_data_wrapper()

另一部分

In [1]:
class Network(object):
    
    #一个[2,3,1]的网络，第一层是输入层，没有biase,剩下两层biase的机构是3*1,2*1的矩阵。
    #weights是两个3*2， 3*1的矩阵。
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x,y in sizes[:-1], sizes[1:]]
    
    # a -> input
    def forward(self, a):
        for b, w in zip(self.weights, self.biases):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, training_data,epochs, mini_batch_size, eta, test_data=None):
        if test_data:n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k : k + mini_batch_size] for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "EPOCH {0} : {1} / {2}".format(j, self.evaluate(test_data), n_test)
            else:
                print "EPOCH {0} complete".format(j)
    
    #
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x,y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.biases = [(b - eta/len(mini_batch)*nb) for b, nb in zip(self.biases, nabla_b)]
        self.weights = [(w - eta/len(mini_batch)*nw) for w, nw in zip(self.weights, nabla_w)]
        
    def backpro(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        #forward
        activation = x
        activations = [x]
        zs = []
        for b,w in zip(self.biase, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            actications.append(activation)
        #backward
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        #此处的layer与普通记号不同，1=the last layer,2 = second last layer...and so no
        #l加了负号，本质是一样的
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(w[l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activation[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)
    
    def sigmoid(z):
        return 1.0/(1.0+np.exp(-z))
    
    def sigmoid_prime(z):
        return sigmoid(z) * (1 - sigmoid(z))